In [29]:
x = 0

In [34]:
import pandas as pd
import glob
import string
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

df = pd.concat(map(pd.read_csv, glob.glob('./coronavirus-covid19-tweets/*2 Coronavirus Tweets.CSV')))

# Remove Tweets in other languages
df = df[df["lang"]=="en"]

# df_processed = df[['user_id','text']].groupby('user_id', as_index = False).agg({'text': 'sum'})

def preprocess_tweet(text):
    # Check characters to see if they are in punctuation
    nopunc = [char for char in text if char not in string.punctuation]
    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    # convert text to lower-case
    nopunc = nopunc.lower()
    # remove URLs
    nopunc = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', nopunc)
    nopunc = re.sub(r'http\S+', '', nopunc)
    # remove usernames
    nopunc = re.sub('@[^\s]+', '', nopunc)
    # remove the # in #hashtag
    nopunc = re.sub(r'#([^\s]+)', r'\1', nopunc)
    #remove numbers
    nopunc = re.sub("^\d+\s|\s\d+\s|\s\d+$"," ", nopunc)
    nopunc = re.sub("\d", "", nopunc)
    # remove repeated characters
    nopunc = re.sub('(corona|covid|virus)', '', nopunc)
    nopunc = word_tokenize(nopunc)
#     return nopunc
    # remove stopwords from final word list
    [word for word in nopunc if word not in stopwords.words('english')]
    text = ' '.join([str(elem) for elem in nopunc]) + "\n"
    return text
#     f.write(' '.join([str(elem) for elem in nopunc]) + "\n")

In [35]:
x += len(df)
print(x)

1870846


In [2]:
# tweet = "RT @marcobonzanini: just an example! #coronavirus :D http://example.com #NLP 1000 p coron6a"
# print(preprocess_tweet(tweet))
# df.head()
len(df)

266973

In [3]:
# with open('tweets1.txt', 'w', encoding="utf8") as f:
#     df.text.map(lambda x:preprocess_tweet(x, f))
df['clean_text'] = df.text.map(lambda x:preprocess_tweet(x))

In [4]:
df.head()

,status_id,user_id,created_at,screen_name,text,source,reply_to_status_id,reply_to_user_id,reply_to_screen_name,is_quote,...,country_code,place_full_name,place_type,followers_count,friends_count,account_lang,account_created_at,verified,lang,clean_text
7,1237891054600241152,3520752864,2020-03-12T00:00:00Z,Tokyo_gov,Watch this newly released video featuring a me...,TweetDeck,NaN,NaN,NaN,False,...,NaN,NaN,NaN,118670,16,NaN,2015-09-11T00:05:21Z,True,en,watch this newly released video featuring a me...
8,1237891054399115264,1056850669,2020-03-12T00:00:00Z,airnewsalerts,"60 positive cases of #Covid19 in India, says H...",TweetDeck,NaN,NaN,NaN,False,...,NaN,NaN,NaN,2236114,13,NaN,2013-01-03T04:15:45Z,True,en,positive cases of in india says health ministry\n
9,1237891053988073473,794162328,2020-03-12T00:00:00Z,StatesideRadio,UM medical historian @HowardMarkel says he doe...,TweetDeck,NaN,NaN,NaN,False,...,NaN,NaN,NaN,3786,412,NaN,2012-08-31T16:08:32Z,False,en,um medical historian howardmarkel says he does...
11,1237891053686075392,38489678,2020-03-12T00:00:00Z,JAMA_current,This JAMA Insights article reviews care for th...,Sprinklr,NaN,NaN,NaN,False,...,NaN,NaN,NaN,335352,805,NaN,2009-05-07T18:45:39Z,True,en,this jama insights article reviews care for th...
12,1237891053627162625,824565311437410305,2020-03-12T00:00:00Z,ViralTabNews,He beat the #coronavirus at age of101 \nhttps:...,TweetDeck,NaN,NaN,NaN,False,...,NaN,NaN,NaN,86,529,NaN,2017-01-26T10:31:04Z,False,en,he beat the at age of\n


In [5]:
df.to_csv(r'ProcessedTweets.csv', index = False)

In [6]:
with open('basilisk-master/ProcessedTweetsTextOnly.txt', 'w', encoding="utf8") as f:
    for val in df['clean_text'].values:
        f.write(val)

In [7]:
# df.to_csv(r'basilisk-master/ProcessedTweetsTextOnly.csv', index = False, columns=['clean_text'])

from nltk.tokenize import TweetTokenizer

inputFolder = "./basilisk-master/out_stressed2/"
tokenizer = TweetTokenizer()
# punct = list(string.punctuation)
# stopwords_additional = ['coronavirus', 'covid19', 'covid-19', 'covid', 'covid_19', '19', 'corona']
# stopwords_list = stopwords.words('english') + punct + ['rt','via','...','…','’','—','—:','“'] + stopwords_additional
# print(stopwords_list)


def read_words(infile):
    words = []
    with open(infile, encoding='utf8') as f:
        for line in f.readlines():
            words.append(line.strip())
    return words

stress_words = read_words(inputFolder + "STRESSED-NS-diffScore.lexicon")            
relax_words = read_words(inputFolder + "NOT_STRESSED-NS-diffScore.lexicon")
# stress_words.extend(read_words(inputFolder + "sample-data/seeds/STRESSED"))

# stress_words = read_words(inputFolder + "sample-data/seeds/STRESSED")            
# relax_words = read_words(inputFolder + "sample-data/seeds/NOT_STRESSED")

# stress_words = ['stress']
# relax_words = ['relax']

# safe_words = read_words(safe_words_file)            
# hazard_words = read_words(hazard_words_file)

def process_tweets(text): 
#     count_safe = 0
#     count_hazard = 0
    count_stress = 0
    count_relax = 0
    text = text.lower()
#     count_stress = text.count('stress')
#     count_relax = text.count('relax')
    tokens = tokenizer.tokenize(text)
#     tokens = [tok for tok in tokens if tok not in stopwords_list and not tok.isdigit()]
#     for tok in tokens:
#         if tok in stress_words:
#             count_stress +=1
#         if tok in relax_words:
#             count_relax +=1
#     if count_stress > count_relax:
#         return 1
#     if count_relax > count_stress:
#         return 2
#     return 0
#     f.write(' '.join([str(elem) for elem in tokens]) + "\n")
#     print(tokens)
#     return count_safe, count_hazard

    for tok in tokens:
        if tok in stress_words:
            return 1
    return 0


def process_tweets_safe(text): 
    count_safe = 0
    count_hazard = 0
    text = text.lower()
    tokens = tokenizer.tokenize(text)
    for tok in tokens:
        if tok in safe_words:
            count_safe +=1
    return count_safe

def process_tweets_hazard(text): 
    count_safe = 0
    count_hazard = 0
    text = text.lower()
    tokens = tokenizer.tokenize(text)
    for tok in tokens:
        if tok in hazard_words:
            count_hazard +=1
    return count_hazard

# for text in df_temp['clean_text'][:100]:
# #     text = text.to_string(index=False)
#     count_safe, count_hazard = process_tweets(text)
# #     print("Safe count: {0}, Hazard count: {1}".format(count_safe, count_hazard))
    
# df_temp["count_safe"] = df_temp.clean_text.map(lambda x:process_tweets_safe(x))
# df_temp["count_hazard"] = df_temp.clean_text.map(lambda x:process_tweets_hazard(x))

# df_kmeans1 = df_temp[["count_safe","count_hazard"]]
# df_kmeans1.head()
# keyword_count_array = np.array(df_kmeans1)
# # px.scatter(df, x="Sentiment", y="Subjectivity",color="cluster_sentiment")
# n_cluster = 5
# pred = KMeans(n_clusters=n_cluster).fit_predict(keyword_count_array)

# # df_temp[['clean_text', 'count_safe', 'count_hazard']].head()
# df_temp["cluster_keyword_count"] = pred
# df_temp.head()


In [ ]:
from nltk.tokenize import TweetTokenizer

inputFolder = "./basilisk-master/out_stressed2/"
tokenizer = TweetTokenizer()
# punct = list(string.punctuation)
# stopwords_additional = ['coronavirus', 'covid19', 'covid-19', 'covid', 'covid_19', '19', 'corona']
# stopwords_list = stopwords.words('english') + punct + ['rt','via','...','…','’','—','—:','“'] + stopwords_additional
# print(stopwords_list)


def read_words(infile):
    words = []
    with open(infile, encoding='utf8') as f:
        for line in f.readlines():
            words.append(line.strip())
    return words

stress_words = read_words(inputFolder + "STRESSED-NS-diffScore.lexicon")            
relax_words = read_words(inputFolder + "NOT_STRESSED-NS-diffScore.lexicon")
# stress_words.extend(read_words(inputFolder + "sample-data/seeds/STRESSED"))

# stress_words = read_words(inputFolder + "sample-data/seeds/STRESSED")            
# relax_words = read_words(inputFolder + "sample-data/seeds/NOT_STRESSED")

# stress_words = ['stress']
# relax_words = ['relax']

# safe_words = read_words(safe_words_file)            
# hazard_words = read_words(hazard_words_file)

def process_tweets(text): 
#     count_safe = 0
#     count_hazard = 0
    count_stress = 0
    count_relax = 0
    text = text.lower()
#     count_stress = text.count('stress')
#     count_relax = text.count('relax')
    tokens = tokenizer.tokenize(text)
#     tokens = [tok for tok in tokens if tok not in stopwords_list and not tok.isdigit()]
    for tok in tokens:
        if tok in stress_words:
            count_stress +=1
        if tok in relax_words:
            count_relax +=1
    if count_stress > count_relax:
        return 1
    if count_relax > count_stress:
        return 2
    return 0
#     f.write(' '.join([str(elem) for elem in tokens]) + "\n")
#     print(tokens)
#     return count_safe, count_hazard

#     for tok in tokens:
#         if tok in stress_words:
#             return 1
#     return 0


def process_tweets_safe(text): 
    count_safe = 0
    count_hazard = 0
    text = text.lower()
    tokens = tokenizer.tokenize(text)
    for tok in tokens:
        if tok in safe_words:
            count_safe +=1
    return count_safe

def process_tweets_hazard(text): 
    count_safe = 0
    count_hazard = 0
    text = text.lower()
    tokens = tokenizer.tokenize(text)
    for tok in tokens:
        if tok in hazard_words:
            count_hazard +=1
    return count_hazard

# for text in df_temp['clean_text'][:100]:
# #     text = text.to_string(index=False)
#     count_safe, count_hazard = process_tweets(text)
# #     print("Safe count: {0}, Hazard count: {1}".format(count_safe, count_hazard))
    
# df_temp["count_safe"] = df_temp.clean_text.map(lambda x:process_tweets_safe(x))
# df_temp["count_hazard"] = df_temp.clean_text.map(lambda x:process_tweets_hazard(x))

# df_kmeans1 = df_temp[["count_safe","count_hazard"]]
# df_kmeans1.head()
# keyword_count_array = np.array(df_kmeans1)
# # px.scatter(df, x="Sentiment", y="Subjectivity",color="cluster_sentiment")
# n_cluster = 5
# pred = KMeans(n_clusters=n_cluster).fit_predict(keyword_count_array)

# # df_temp[['clean_text', 'count_safe', 'count_hazard']].head()
# df_temp["cluster_keyword_count"] = pred
# df_temp.head()
# 

In [16]:
df['stress_label'] = df.clean_text.map(lambda x:process_tweets(x))
# df['stress_label2'] = df.clean_text.map(lambda x:process_tweets(x))

In [17]:
# df.to_csv(r'ProcessedTweetsWithStressLabels.csv', index=True)

In [18]:
# df['stress_label_new'] = df.clean_text.map(lambda x:process_tweets(x))

In [19]:
# df.columns

In [20]:
# df[['clean_text', 'stress_label']].head()
# print(df[df['stress_label']==2][['clean_text', 'stress_label']]['clean_text'].values)

In [21]:
X_train_df = df[df['stress_label']>=1]
import tensorflow as tf
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [22]:
# cfg = tf.ConfigProto(allow_soft_placement=True )
# cfg.gpu_options.allow_growth = True
# with tf.Session(config=cfg) as session:
#     session.run([tf.global_variables_initializer(), tf.tables_initializer()])
#     message_embeddings = session.run(embed(X_train_df['clean_text'].values))
# #     message_embeddings = session.run(embed(list(user_tweets.values())))
# print(message_embeddings.shape)

In [23]:
def get_chunk(df):
    data = df["clean_text"].tolist()
    chunks_size = 500
    chunks = [data[x : x + chunks_size] for x in range(0, len(data), chunks_size)]
    print(len(chunks))
    return chunks

# chunks = get_chunk(df)



In [24]:
import tensorflow as tf
print(tf.__version__)
cfg = tf.compat.v1.ConfigProto(allow_soft_placement=True )
cfg.gpu_options.allow_growth = True

def get_embedding(chunks):
    message_embeddings = []
    with tf.Session(config=cfg) as session:
        for index, ck in enumerate(chunks):
            session.run([tf.global_variables_initializer(), tf.tables_initializer()])
            # if index == 1:
            #   break
#             message_embeddings.append(session.run(embed(ck)))
            message_embeddings.extend(session.run(embed(ck)))
            print("finished ", index)
            # message_embeddings = session.run(embed(list(user_tweets.values())))
    return message_embeddings
# message_embeddings = get_embedding(chunks)

1.15.0


In [25]:
print(len(df[df['stress_label']==1]))
print(len(df[df['stress_label']==2]))
# print(len(df[df['stress_label2']==2]))

235966


In [ ]:
chunks_stress = get_chunk(df[df['stress_label2']==1][:10000])
chunks_relax = get_chunk(df[df['stress_label2']==0][:10000])
stress_text_embeddings = get_embedding(chunks_stress)
relax_text_embeddings = get_embedding(chunks_relax)


In [ ]:
import numpy as np
X = []
for embedding in stress_text_embeddings:  
    X.append(embedding)
for embedding in relax_text_embeddings:  
    X.append(embedding)
# relax_text_embeddings.shape
len(X)
X = np.asarray(X)

In [ ]:
def get_text_embedding(text_list):
    with tf.Session(config=cfg) as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        return session.run(embed(text_list))

# stress_text_embeddings = get_text_embedding(df[df['stress_label_new']==1]['clean_text'].values)
# relax_text_embeddings = get_text_embedding(df[df['stress_label_new']==2]['clean_text'].values)



In [ ]:
len(X)

In [ ]:
# print(stress_text_embeddings.shape)
# print(relax_text_embeddings.shape)
import random
from sklearn.model_selection import train_test_split
import numpy as np

# X = np.concatenate((stress_text_embeddings[:250], relax_text_embeddings[:250]), axis=0)
# X = stress_text_embeddings[:250]
# np.append(X, relax_text_embeddings[:250])
# y = np.ones((250,), dtype=int)
# np.append(y, np.zeros((250,), dtype=int))
ones = np.ones((10000,), dtype=int)
zeros = np.zeros((10000,), dtype=int)
y = np.concatenate((ones,zeros), axis=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

# X_val = np.concatenate((stress_text_embeddings, relax_text_embeddings), axis=0)
# y_val = np.concatenate((np.ones((len(stress_text_embeddings),), dtype=int),np.zeros((len(relax_text_embeddings),), dtype=int)), axis=0)
# c = list(zip(X_val, y_val))

# random.shuffle(c)

# X_val, y_val = zip(*c)
# X_val, _, y_val, _ = train_test_split(X_val, y_val, test_size=0, random_state=42, shuffle=True)

# stress_text_embeddings_labeled = [(embedding, 1) for embedding in stress_text_embeddings]
# relax_text_embeddings_labeled = [(embedding, 1) for embedding in relax_text_embeddings]
# text_embeddings_labeled = []
# text_embeddings_labeled.extend(stress_text_embeddings_labeled)
# text_embeddings_labeled.extend(relax_text_embeddings_labeled)
# # random.shuffle(text_embeddings_labeled)
# print(text_embeddings_labeled[:20])


In [ ]:
from sklearn import svm

svc = svm.SVC(kernel='linear', C=1,gamma='auto')
svc.fit(X_train, y_train)
print("Accuracy for train: ", svc.score(X_train, y_train))
print("Accuracy for test: ", svc.score(X_test, y_test))
# print("Accuracy for stressed: ", svc.score(stress_text_embeddings, np.ones((len(stress_text_embeddings),), dtype=int)))
# print("Accuracy for relaxed: ", svc.score(relax_text_embeddings, np.zeros((len(relax_text_embeddings),), dtype=int)))
# print("Accuracy for mixed: ", svc.score(X_val, y_val))




# svclassifier = SVC(C=10, probability=True, gamma='scale', kernel='linear')
# X_train, X_test, y_train, y_test =   cross_validation.train_test_split(iris.data,   iris.target, test_size=0.10, random_state=111)

In [ ]:
from sklearn import svm

svc = svm.SVC(kernel='rbf', C=1,gamma='auto')
svc.fit(X_train, y_train)
print("Accuracy for train: ", svc.score(X_train, y_train))
print("Accuracy for test: ", svc.score(X_test, y_test))

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=0)
lr.fit(X_train, y_train)
print("Accuracy for train: ", lr.score(X_train, y_train))
print("Accuracy for test: ", lr.score(X_test, y_test))

In [ ]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(512,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=40, batch_size=512)
results = model.evaluate(X_test, y_test)
results

# model = models.Sequential()
# model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
# model.add(layers.Dense(16, activation='relu'))
# model.add(layers.Dense(1, activation='sigmoid'))
# model.compile(optimizer='rmsprop',
#               loss='binary_crossentropy',
#               metrics=['accuracy'])
# history = model.fit(partial_x_train,
#                     partial_y_train,
#                     epochs=20,
#                     batch_size=512,
#                     validation_data=(x_val, y_val))
# history_dict = history.history
# history_dict.keys()
# import matplotlib.pyplot as plt

# acc = history.history['acc']
# val_acc = history.history['val_acc']
# loss = history.history['loss']
# val_loss = history.history['val_loss']

# epochs = range(1, len(acc) + 1)

# # "bo" is for "blue dot"
# plt.plot(epochs, loss, 'bo', label='Training loss')
# # b is for "solid blue line"
# plt.plot(epochs, val_loss, 'b', label='Validation loss')
# plt.title('Training and validation loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()

# plt.show()

In [ ]:
# y_pred = model.predict(get_text_embedding(df['clean_text'].values))
# chunks = get_chunk(df[:50000])
chunks = get_chunk(df)

text_embeddings = get_embedding(chunks)

In [ ]:
model.save('tweets_model2.h5')

In [ ]:
len(df_labeled[df_labeled['predicted_prob']<0.5])

In [ ]:
X = np.asarray(text_embeddings)
X.shape
# y_pred = model.predict(text_embeddings)


In [ ]:
# y_pred = model.predict(get_text_embedding(df[0]['clean_text'].values))
# y_pred.shape
y_pred = model.predict(X)
y_list = [pred[0] for pred in y_pred]
print(len(y_list))
df_labeled = df[:50000]
print(len(df_labeled))
df_labeled['predicted_prob'] = y_list
df_labeled.head()
print(df_labeled[df_labeled['predicted_prob']>0.5]['clean_text'].values[:50])
# y_list_stress = [pred[0] for pred in y_pred if pred[0]>]
# y_pred.tolist()

In [ ]:
# from collections import Counter 
# # wordcloud in python
# from wordcloud import WordCloud, STOPWORDS 
# import matplotlib.pyplot as plt

# import re 
# import string
# import nltk # preprocessing text
# from textblob import TextBlob

# wordcloud = WordCloud(width=1600, height=800, random_state=1, max_words=100,colormap="Paired", background_color='black',)
# wordcloud.generate(str(df_labeled[df_labeled['predicted_prob']>0.5]['clean_text']))
# # declare our figure 
# plt.figure(figsize=(20,10))
# plt.title("Top words - tweets", fontsize=20,color='black')
# plt.imshow(wordcloud, interpolation='bilinear')
# plt.axis('off')
# plt.tight_layout(pad=10)
# plt.show()

In [ ]:
y_labels_list = []
for prob in y_list:
    if prob >= 0.5:
        y_labels_list.append(1)
    else:
        y_labels_list.append(0)
df_labeled['predicted_label'] = y_labels_list

df_labeled.head()


In [ ]:
from sklearn.linear_model import LogisticRegression
lr_2 = LogisticRegression(random_state=0)
lr_2.fit(X, y_labels_list)
print("Accuracy for train: ", lr.score(X, y_labels_list))
# print("Accuracy for test: ", lr.score(X_test, y_test))


In [ ]:
model.save('tweets_model3.h5')

In [ ]:
# lr_2.predict(embed("I am sad")
# y_pred2 = lr_2.predict_proba(get_text_embedding(["There is too much work and no time"]))
y_pred2 = lr_2.predict(X)
df_labeled['lr_label'] = y_pred2.tolist()
print(len(df_labeled[df_labeled['lr_label']==1]))
print(df_labeled[df_labeled['lr_label']==1]['text'].values)

# print(y_pred2.tolist())
# print(df_labeled['clean_text'].values[:21])
# df_labeled2 = df_labeled[:50000]
# print(len(df_labeled))
# df_labeled['predicted_prob'] = y_list
# df_labeled.head()
# print(df_labeled[df_labeled['predicted_prob']>0.5]['clean_text'].values[:50])
# y_list2 = [pred[0] for pred in y_pred2]


In [ ]:
with open('stress_classified_tweets.txs', 'w', encoding="utf8") as f:
    for val in df_labeled[df_labeled['lr_label']==1]['text']:
        f.write(val +"\n")

In [ ]:
print(df['clean_text'].values[:20])
print(y_pred)

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
print(len(X_train))
print("Accuracy for train: ", gnb.score(X_train, y_train))
print("Accuracy for test: ", gnb.score(X_test, y_test))
# y_pred = gnb.fit(X_train, y_train).predict(X_test)

In [ ]:
df_processed = df[:50000].copy()
df_processed = df_processed[df_processed["lang"] == "en"]
df_processed.shape

In [ ]:
df_processed['pro_text'] = df_processed.apply(lambda row: preprocess_tweet(row['text']), axis = 1)

In [ ]:
df_processed.head()

In [ ]:
# import numpy as np
# dimensions = 200

# # Get embeddings
# pretrained_vectors = {}
# f = open('./glove.twitter.27B/glove.twitter.27B.200d.txt', encoding="utf8")
# for line in f:
#     values = line.rstrip().rsplit(' ', dimensions)
#     word = values[0]
#     probabilities = np.asarray(values[1:], dtype='float32')
#     pretrained_vectors[word] = probabilities
# f.close()

# print('Found {} word vectors.'.format(len(pretrained_vectors)))

In [ ]:
# print(pretrained_vectors['tweet'])

In [ ]:
df_processed = df_processed[['user_id','pro_text']].groupby('user_id', as_index = False).agg({'pro_text': 'sum'})

In [ ]:
df_processed.shape

In [ ]:
df_processed['pro_text']=[" ".join(review) for review in df_processed['pro_text'].values]

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [ ]:
tfidf = TfidfVectorizer(
    min_df = 5,
    max_df = 0.95,
    max_features = 8000,
    stop_words = 'english'
)
tfidf.fit(df_processed.pro_text)
text = tfidf.transform(df_processed.pro_text)

In [ ]:
from sklearn.cluster import MiniBatchKMeans

def find_optimal_clusters(data, max_k):
    iters = range(2, max_k+1, 2)
    
    sse = []
    for k in iters:
        sse.append(MiniBatchKMeans(n_clusters=k, init_size=1024, batch_size=2048, random_state=20).fit(data).inertia_)
        print('Fit {} clusters'.format(k))
        
    f, ax = plt.subplots(1, 1)
    ax.plot(iters, sse, marker='o')
    ax.set_xlabel('Cluster Centers')
    ax.set_xticks(iters)
    ax.set_xticklabels(iters)
    ax.set_ylabel('SSE')
    ax.set_title('SSE by Cluster Center Plot')
    
find_optimal_clusters(text, 40)

In [ ]:
def plot_tsne_pca(data, labels):
    max_label = max(labels)
    max_items = np.random.choice(range(data.shape[0]), size=3000, replace=False)
    
    pca = PCA(n_components=2).fit_transform(data[max_items,:].todense())
    tsne = TSNE().fit_transform(PCA(n_components=50).fit_transform(data[max_items,:].todense()))
    
    
    idx = np.random.choice(range(pca.shape[0]), size=300, replace=False)
    label_subset = labels[max_items]
    label_subset = [cm.hsv(i/max_label) for i in label_subset[idx]]
    
    f, ax = plt.subplots(1, 2, figsize=(14, 6))
    
    ax[0].scatter(pca[idx, 0], pca[idx, 1], c=label_subset)
    ax[0].set_title('PCA Cluster Plot')
    
    ax[1].scatter(tsne[idx, 0], tsne[idx, 1], c=label_subset)
    ax[1].set_title('TSNE Cluster Plot')
    
clusters = MiniBatchKMeans(n_clusters=5, init_size=1024, batch_size=2048, random_state=20).fit_predict(text)
plot_tsne_pca(text, clusters)

In [ ]:
def get_top_keywords(data, clusters, labels, n_terms):
    df = pd.DataFrame(data.todense()).groupby(clusters).mean()
    
    for i,r in df.iterrows():
        print('\nCluster {}'.format(i))
        print(','.join([labels[t] for t in np.argsort(r)[-n_terms:]]))
            
get_top_keywords(text, clusters, tfidf.get_feature_names(), 10)